# Optimization SIRDS simulations

In [1]:
import numpy as np

from sirds_model import brazil_covid_sirds_model, get_new_deaths, get_effective_reproduction_number
import pandas as pd
from sklearn.metrics import mean_absolute_error
from scipy.optimize import differential_evolution
from datetime import timedelta
import csv

## Reading data

In [2]:
df = pd.read_csv('data/output/df_ts_covid_brazil.csv')
df.DATA = pd.to_datetime(df.DATA, infer_datetime_format=True)

In [3]:
valleys = pd.read_csv('/data/output/valleys.csv', header=None)
valleys.iloc[:,0] = pd.to_datetime(valleys.iloc[:,0], infer_datetime_format=True)
valleys = valleys.iloc[:,0].values
valley_1 = valleys[0]
valley_2 = valleys[1]
valley_3 = valleys[2]

peaks = pd.read_csv('/data/output/peaks.csv',header=None)
peaks.iloc[:,0] = pd.to_datetime(peaks.iloc[:,0], infer_datetime_format=True)
peaks = peaks.iloc[:,0].values
peak_1 = peaks[0]
peak_2 = peaks[1]
peak_3 = peaks[2]
peak_4 = peaks[3]

date_max = df.DATA.max()

## Preparing data

In [4]:
df = df.sort_values(by='DATA')

In [5]:
df

,DATA,SIGLA_ESTADO,MUNICIPIO,CODIGO_MUNICIPIO_7,OBITOS_NOVOS,OBITOS,CASOS_NOVOS,CASOS,POPULACAO_ESTIMADA,OBITOS_NOVOS_MEDIA_14_DIAS,...,NUMERO_REPRODUCAO_EFETIVO_MEDIA,NUMERO_REPRODUCAO_EFETIVO_VARIANCIA,NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.025,NUMERO_REPRODUCAO_EFETIVO_MEDIANA,NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.975,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIA,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_VARIANCIA,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_QUANTIL_0.025,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_MEDIANA,NUMERO_REPRODUCAO_EFETIVO_ATRASADO_QUANTIL_0.975
0,2020-02-25,TOTAL,TOTAL,0,0,0,1,1,210147125,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-26,TOTAL,TOTAL,0,0,0,0,1,210147125,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-27,TOTAL,TOTAL,0,0,0,0,1,210147125,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-28,TOTAL,TOTAL,0,0,0,1,2,210147125,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-29,TOTAL,TOTAL,0,0,0,0,2,210147125,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,2022-10-11,TOTAL,TOTAL,0,86,686963,7350,34731539,210147125,77.285714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
960,2022-10-12,TOTAL,TOTAL,0,63,687026,5114,34736653,210147125,78.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
961,2022-10-13,TOTAL,TOTAL,0,43,687069,3212,34739865,210147125,77.928571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,2022-10-14,TOTAL,TOTAL,0,75,687144,6597,34746462,210147125,79.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preparing parameters and bounds

### Days to recovery

In [6]:
# Considering recovery period of 14 days for mild cases as reported by “Report of the WHO-China Joint Mission on Coronavirus Disease 2019 (COVID-19).” Accessed September 26, 2022. https://www.who.int/publications-detail-redirect/report-of-the-who-chinajoint-mission-on-coronavirus-disease-2019-(covid-19).
# Considering recovery period mean between 13 and 15 second Voinsky, Irena, Gabriele Baristaite, and David Gurwitz. "Effects of age and sex on recovery from COVID-19: Analysis of 5769 Israeli patients." Journal of Infection 81.2 (2020): e102-e103.
    # Also using 14 days as mean +/- 6 SD we have range 8-20 or 8-26 if we considerate incubation period
# Considerering incubation period around 6 days as reported by Bi et al (2020). Epidemiology and transmission of covid-19 in 391 cases and 1286 of their close contacts in shenzhen, china: a retrospective cohort study. The Lancet infectious diseases, 20(8):911–919, 2020.
list_days_to_recovery = [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
# list_days_to_recovery = [8]

### Breakpoints
We consider breakpoints the valleys between case waves.

In [44]:
df.DATA.max()

Timestamp('2022-10-15 00:00:00')

In [7]:
date_first_case = df[(df.TAXA_CASOS_NOVOS_MEDIA_14_DIAS > 0)].iloc[0].DATA
date_first_case

Timestamp('2020-02-25 00:00:00')

In [8]:
date_first_case + timedelta(days=21)

Timestamp('2020-03-17 00:00:00')

In [9]:
wave_inner_breakpoint_1 = ((date_first_case + timedelta(days=21)) - date_first_case).days

wave_inner_breakpoint_1

21

In [10]:
breakpoint_1 = (valley_1 - date_first_case).days
breakpoint_1

258

In [11]:
breakpoint_2 = (valley_2 - date_first_case).days
breakpoint_2

668

In [12]:
breakpoint_3 = (valley_3 - date_first_case).days
breakpoint_3

793

In [13]:
min_wave_inner_breakpoint_1 = ((date_first_case + timedelta(days=1)) - date_first_case).days

min_wave_inner_breakpoint_1

1

In [14]:
max_wave_inner_breakpoint_1 = (peak_1 - date_first_case).days

max_wave_inner_breakpoint_1

161

In [15]:
min_breakpoint_1 = breakpoint_1 - (breakpoint_1 * 0.05)
min_breakpoint_1

245.1

In [16]:
max_breakpoint_1 = breakpoint_1 + (breakpoint_1 * 0.05)
max_breakpoint_1

270.9

In [17]:
min_breakpoint_2 = breakpoint_2 - (breakpoint_2 - breakpoint_1) * 0.05
min_breakpoint_2

647.5

In [18]:
max_breakpoint_2 = breakpoint_2 + (breakpoint_2 - breakpoint_1) * 0.05
max_breakpoint_2

688.5

In [19]:
min_breakpoint_3 = breakpoint_3 - (breakpoint_3 - breakpoint_2) * 0.05
min_breakpoint_3

786.75

In [20]:
max_breakpoint_3 = breakpoint_3 + (breakpoint_3 - breakpoint_2) * 0.05
max_breakpoint_3

799.25

### Period of analysis

In [21]:
date_begin_time_series = df.iloc[0].DATA
t_first_case = (date_first_case - date_begin_time_series).days
period_in_days = len(df) - t_first_case
period_in_days

964

### Days between infections
We infer the days between infectiosn based on effective reproduction number (Rt)

In [22]:
def get_days_between_infections_0(days_to_recovery):
    R0_mean = df['NUMERO_REPRODUCAO_EFETIVO_MEDIA'].dropna().iloc[0]
    R0_min = df['NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.025'].dropna().iloc[0]
    R0_max = df['NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.975'].dropna().iloc[0]

    days_between_infections_0_min = days_to_recovery/5.5

    # Best result until now
    # days_between_infections_0_max = days_to_recovery/1.9
    days_between_infections_0_max = days_to_recovery/1

    days_between_infections_default = days_to_recovery/R0_mean

    return days_between_infections_0_min, days_between_infections_default, days_between_infections_0_max

In [23]:
def get_days_between_infections_in_wave(date_begin, date_end, days_to_recovery):
    df_wave = df[(df.DATA > date_begin) & (df.DATA <= date_end)]
    date_max_rt = df_wave[df_wave.NUMERO_REPRODUCAO_EFETIVO_MEDIA == df_wave.NUMERO_REPRODUCAO_EFETIVO_MEDIA.max()].DATA.min()
    R_min = df_wave[df_wave.DATA == date_max_rt]['NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.025'].iloc[0]
    R_max = df_wave[df_wave.DATA == date_max_rt]['NUMERO_REPRODUCAO_EFETIVO_QUANTIL_0.975'].iloc[0]
    R_mean = df_wave[df_wave.DATA == date_max_rt]['NUMERO_REPRODUCAO_EFETIVO_MEDIA'].iloc[0]

    days_between_infections_min = days_to_recovery/10
    days_between_infections_max = days_to_recovery/1
    days_between_infections_default = days_to_recovery/R_mean

    return days_between_infections_min, days_between_infections_default, days_between_infections_max

def get_days_between_infections_in_changed_wave(days_to_recovery):
    days_between_infections_min = days_to_recovery/1.47
    days_between_infections_max = days_to_recovery/1
    days_between_infections_default = days_between_infections_min + (days_between_infections_max - days_between_infections_min)/2

    return days_between_infections_min, days_between_infections_default, days_between_infections_max

### Case fatality probabilities
For maximum bound, we use the ratio between deaths and cases in the period.
For minimum bound we consider an extreme limit like whether all Brazilians had been infected by Covid-19 during the period.

In [24]:
populcao_brasileira = df.POPULACAO_ESTIMADA[0]
populcao_brasileira

210147125

In [25]:
# Best result until now
min_case_fatality_probability_0 = df[df.DATA <= valley_2].OBITOS_NOVOS.sum()/populcao_brasileira
min_case_fatality_probability_0

0.002942942950087944

In [26]:
# Best result until now
min_case_fatality_probability_1 = df[df.DATA <= valley_2].OBITOS_NOVOS.sum()/populcao_brasileira
min_case_fatality_probability_1

0.002942942950087944

In [27]:
# Best result until now
min_case_fatality_probability_2 = df[(df.DATA > valley_2)].OBITOS_NOVOS.sum()/populcao_brasileira
min_case_fatality_probability_2

0.00032713271713805266

In [28]:
# Best result until now
min_case_fatality_probability_3 = df[(df.DATA > valley_2)].OBITOS_NOVOS.sum()/populcao_brasileira
min_case_fatality_probability_3

0.00032713271713805266

In [50]:
df[df.DATA <= valley_1].OBITOS_NOVOS.sum()/df[df.DATA <= valley_1].CASOS_NOVOS.sum()*100

2.8643130009176687

In [51]:
df[(df.DATA > valley_1) & (df.DATA <= valley_2)].OBITOS_NOVOS.sum()/df[(df.DATA > valley_1) & (df.DATA <= valley_2)].CASOS_NOVOS.sum()*100

2.7532668983600006

In [29]:
# Best result until now
max_case_fatality_probability_0 = df[df.DATA <= valley_2].OBITOS_NOVOS.sum()/df[df.DATA <= valley_2].CASOS_NOVOS.sum()
max_case_fatality_probability_0

0.027816395981419963

In [30]:
# Best result until now
max_case_fatality_probability_1 = df[df.DATA <= valley_2].OBITOS_NOVOS.sum()/df[df.DATA <= valley_2].CASOS_NOVOS.sum()
max_case_fatality_probability_1

0.027816395981419963

In [31]:
# Best result until now
max_case_fatality_probability_2 = df[(df.DATA > valley_2)].OBITOS_NOVOS.sum()/df[(df.DATA > valley_2)].CASOS_NOVOS.sum()
max_case_fatality_probability_2

0.005492212787494123

In [32]:
# Best result until now
max_case_fatality_probability_3 = df[(df.DATA > valley_2)].OBITOS_NOVOS.sum()/df[(df.DATA > valley_2)].CASOS_NOVOS.sum()
max_case_fatality_probability_3

0.005492212787494123

### Loss immunity

In [33]:
# Pulliam, Juliet RC, et al. "Increased risk of SARS-CoV-2 reinfection associated with emergence of Omicron in South Africa." Science 376.6593 (2022): eabn4947.
# This work report reinfection cases 90 days after first infection. This work also show reinfection peaks 170, 350, and 520 days after last infection.
default_loss_immunity_in_days = 180
default_loss_immunity_in_days

180

In [34]:
min_loss_immunity_in_days = 90
min_loss_immunity_in_days

90

In [35]:
max_loss_immunity_in_days = 365
max_loss_immunity_in_days

365

### Number of initial infections in Brazil

In [36]:
min_initial_infected = df[(df.TAXA_CASOS_NOVOS_MEDIA_14_DIAS > 0)].iloc[0].TAXA_CASOS_NOVOS_MEDIA_14_DIAS
min_initial_infected

0.00047585709297712263

In [37]:
max_initial_infected = df[df.DATA == date_first_case + timedelta(days=21)].CASOS.values[0] / df.POPULACAO_ESTIMADA[0] * 100000
max_initial_infected

0.1584604119613818

In [38]:
df[df.DATA == date_first_case + timedelta(days=21)].CASOS.values[0]

333

### Setting bounds

In [39]:
ruido = 0.000001

def create_bound(min, max):
    return (min - ruido, max + ruido)

In [40]:
def get_bounds(days_to_recovey):
    bounds_initial_infected_population = create_bound(min_initial_infected, max_initial_infected)

    bounds_loss_immunity_in_days = create_bound(min_loss_immunity_in_days, max_loss_immunity_in_days)

    min_days_between_infections_0, _, max_days_between_infections_0 = get_days_between_infections_0(days_to_recovey)
    bounds_days_between_infections_0 = create_bound(min_days_between_infections_0, max_days_between_infections_0)
    bounds_case_fatality_probability_0 = create_bound(min_case_fatality_probability_0, max_case_fatality_probability_0)

    min_days_between_infections, _, max_days_between_infections = get_days_between_infections_in_wave(valley_1, peak_2, days_to_recovey)
    bounds_days_between_infections_1 = create_bound(min_days_between_infections, max_days_between_infections)
    bounds_breakpoint_1 = create_bound(min_breakpoint_1, max_breakpoint_1)

    min_days_between_infections, _, max_days_between_infections = get_days_between_infections_in_wave(valley_2, peak_3, days_to_recovey)
    bounds_days_between_infections_2 = create_bound(min_days_between_infections, max_days_between_infections)
    bounds_breakpoint_2 = create_bound(min_breakpoint_2, max_breakpoint_2)
    bounds_case_fatality_probability_2 = create_bound(min_case_fatality_probability_2, max_case_fatality_probability_2)

    min_days_between_infections, _, max_days_between_infections = get_days_between_infections_in_wave(valley_3, peak_4, days_to_recovey)
    bounds_days_between_infections_3 = create_bound(min_days_between_infections, max_days_between_infections)
    bounds_breakpoint_3 = create_bound(min_breakpoint_3, max_breakpoint_3)

    bounds_wave_inner_breakpoint_1 = create_bound(min_wave_inner_breakpoint_1, max_wave_inner_breakpoint_1)
    min_days_between_infections, _, max_days_between_infections = get_days_between_infections_in_changed_wave(days_to_recovey)
    bound_days_between_infections_wave_inner_1 = create_bound(min_days_between_infections, max_days_between_infections)

    bounds = [
        bounds_initial_infected_population,
        bounds_case_fatality_probability_0,
        bounds_days_between_infections_0,
        bounds_loss_immunity_in_days,
        bounds_breakpoint_1,
        bounds_days_between_infections_1,
        bounds_loss_immunity_in_days,
        bounds_breakpoint_2,
        bounds_case_fatality_probability_2,
        bounds_days_between_infections_2,
        bounds_loss_immunity_in_days,
        bounds_breakpoint_3,
        bounds_days_between_infections_3,
        bounds_wave_inner_breakpoint_1,
        bound_days_between_infections_wave_inner_1
    ]

    return bounds

### Setting initial parameters

In [41]:
def get_x0(days_to_recovery):
    _, days_between_infections_0, _ = get_days_between_infections_0(days_to_recovery)
    _, days_between_infections_1, _ = get_days_between_infections_in_wave(valley_1, peak_2, days_to_recovery)
    _, days_between_infections_2, _ = get_days_between_infections_in_wave(valley_2, peak_3, days_to_recovery)
    _, days_between_infections_3, _ = get_days_between_infections_in_wave(valley_3, peak_4, days_to_recovery)
    _, days_between_infections_inner_wave_1, _ = get_days_between_infections_in_changed_wave(days_to_recovery)

    x0 = [min_initial_infected,
      max_case_fatality_probability_0, days_between_infections_0, default_loss_immunity_in_days,
      breakpoint_1, days_between_infections_1, default_loss_immunity_in_days,
      breakpoint_2, max_case_fatality_probability_2, days_between_infections_2, default_loss_immunity_in_days,
      breakpoint_3, days_between_infections_3, wave_inner_breakpoint_1, days_between_infections_inner_wave_1]

    return x0

### Optimization

In [42]:
def get_error_deaths_rt(SIRDS_new_deaths, real_new_deaths, SIRDS_effective_reproduction_number, real_effective_reproduction_number):
    mae_deaths = mean_absolute_error(real_new_deaths[1:], SIRDS_new_deaths)

    indices_to_remove = np.argwhere(np.isnan(real_effective_reproduction_number))
    real_effective_reproduction_number = np.delete(real_effective_reproduction_number, indices_to_remove)
    SIRDS_effective_reproduction_number = np.delete(SIRDS_effective_reproduction_number, indices_to_remove)

    mae_rt = mean_absolute_error(real_effective_reproduction_number, SIRDS_effective_reproduction_number)

    error = mae_deaths/real_new_deaths[1:].mean() + mae_rt/ real_effective_reproduction_number.mean()

    return error

def get_loss_immunity_in_days_3(loss_immunity_in_days_0, loss_immunity_in_days_1, loss_immunity_in_days_2):
    loss_immunity_in_days_3 = (loss_immunity_in_days_0 + loss_immunity_in_days_1 + loss_immunity_in_days_2) / 3
    return loss_immunity_in_days_3

def optimization_brazil_covid_sirds_model(x, *args):
    initial_infected_population = x[0]

    case_fatality_probability_0 = x[1]
    days_between_infections_0 = x[2]
    loss_immunity_in_days_0 = x[3]

    breakpoint_1 = x[4]
    days_between_infections_1 = x[5]
    loss_immunity_in_days_1 = x[6]

    breakpoint_2 = x[7]
    case_fatality_probability_2 = x[8]
    days_between_infections_2 = x[9]
    loss_immunity_in_days_2 = x[10]

    breakpoint_3 = x[11]
    days_between_infections_3 = x[12]

    wave_inner_breakpoint_1 = x[13]
    days_between_infections_inner_wave_1 = x[14]

    loss_immunity_in_days_3 = get_loss_immunity_in_days_3(loss_immunity_in_days_0, loss_immunity_in_days_1, loss_immunity_in_days_2)

    period_in_days = args[0]
    real_new_deaths = args[1]
    case_evolution_data = args[2]
    days_to_recovery = args[3]

    total_population = 100000

    y = brazil_covid_sirds_model(initial_infected_population=initial_infected_population, period_in_days=period_in_days, days_between_infections_0=days_between_infections_0, days_to_recovery_0=days_to_recovery, case_fatality_probability_0=case_fatality_probability_0, loss_immunity_in_days_0=loss_immunity_in_days_0, breakpoint_1=breakpoint_1, days_between_infections_1=days_between_infections_1, days_to_recovery_1=days_to_recovery, case_fatality_probability_1=case_fatality_probability_0, loss_immunity_in_days_1=loss_immunity_in_days_1, breakpoint_2=breakpoint_2, days_between_infections_2=days_between_infections_2, days_to_recovery_2=days_to_recovery, case_fatality_probability_2=case_fatality_probability_2, loss_immunity_in_days_2=loss_immunity_in_days_2, breakpoint_3=breakpoint_3, days_between_infections_3=days_between_infections_3, days_to_recovery_3=days_to_recovery, case_fatality_probability_3=case_fatality_probability_2, loss_immunity_in_days_3=loss_immunity_in_days_3, total_population=total_population, wave_inner_breakpoint_1=wave_inner_breakpoint_1, days_between_infections_inner_wave_1=days_between_infections_inner_wave_1)

    S, I, R, D, I_accumulated = y
    N = S[0] + I[0] + R[0] + D[0]

    SIRDS_new_deaths = get_new_deaths(D)

    SIRDS_effective_reproduction_number = get_effective_reproduction_number(S, N, days_to_recovery, days_between_infections_0, breakpoint_1, days_to_recovery, days_between_infections_1, breakpoint_2, days_to_recovery, days_between_infections_2, breakpoint_3, days_to_recovery, days_between_infections_3, wave_inner_breakpoint_1, days_between_infections_inner_wave_1)
    return get_error_deaths_rt(SIRDS_new_deaths, real_new_deaths, SIRDS_effective_reproduction_number, case_evolution_data)

In [43]:
real_new_deaths = df[df.DATA >= date_first_case].TAXA_OBITOS_NOVOS_MEDIA_14_DIAS.round(6).values

case_evolution_data = df[df.DATA >= date_first_case]['NUMERO_REPRODUCAO_EFETIVO_MEDIA'].round(6).values

for days_to_recovery in list_days_to_recovery:
    print('days_to_recovery: '+str(days_to_recovery))

    bounds = get_bounds(days_to_recovery)
    x0 = get_x0(days_to_recovery)

    df_optimization_result = pd.DataFrame(columns=['error', 'number_evaluations', 'number_iterations', 'initial_infected_population', 'period_in_days', 'days_between_infections_0', 'days_to_recovery_0', 'case_fatality_probability_0', 'loss_immunity_in_days_0', 'breakpoint_1', 'days_between_infections_1', 'days_to_recovery_1', 'case_fatality_probability_1', 'loss_immunity_in_days_1', 'breakpoint_2', 'days_between_infections_2', 'days_to_recovery_2', 'case_fatality_probability_2', 'loss_immunity_in_days_2', 'breakpoint_3', 'days_between_infections_3', 'days_to_recovery_3', 'case_fatality_probability_3', 'loss_immunity_in_days_3', 'wave_inner_breakpoint_1', 'days_between_infections_inner_wave_1'])

    args = [period_in_days, real_new_deaths, case_evolution_data, days_to_recovery]

    for i in range(5):
        print('Iteração: '+str(i))

        result = differential_evolution(optimization_brazil_covid_sirds_model, bounds, x0=x0, args=args, maxiter=1000, workers=7, updating='deferred')

        loss_immunity_in_days_3 = get_loss_immunity_in_days_3(result.x[3], result.x[7], result.x[11])

        print('Result')

        print(result.fun)

        df_optimization_result = df_optimization_result.append({'error': result.fun, 'number_evaluations': result.nfev, 'number_iterations': result.nit, 'period_in_days': period_in_days, 'initial_infected_population': result.x[0],'case_fatality_probability_0' : result.x[1],'days_to_recovery_0' : days_to_recovery,'days_between_infections_0' : result.x[2],'loss_immunity_in_days_0' : result.x[3], 'breakpoint_1' : result.x[4],'case_fatality_probability_1' : result.x[1],'days_to_recovery_1' : days_to_recovery,'days_between_infections_1' : result.x[5],'loss_immunity_in_days_1' : result.x[6], 'breakpoint_2' : result.x[7],'case_fatality_probability_2' : result.x[8],'days_to_recovery_2' : days_to_recovery,'days_between_infections_2' : result.x[9],'loss_immunity_in_days_2' : result.x[10],'breakpoint_3' : result.x[11],'case_fatality_probability_3' : result.x[8],'days_to_recovery_3' : days_to_recovery,'days_between_infections_3' : result.x[12],'loss_immunity_in_days_3' : loss_immunity_in_days_3, 'wave_inner_breakpoint_1': result.x[13], 'days_between_infections_inner_wave_1': result.x[14]}, ignore_index=True)

        df_optimization_result.to_csv('data/output/sirds/df_differential_evolution_national_days_to_recovery_'+str(days_to_recovery)+'.csv', index=False)
    with open('data/output/sirds/bounds_'+str(days_to_recovery)+'.csv', 'w') as myfile:
        write = csv.writer(myfile)
        write.writerows(bounds)

days_to_recovery: 8
Iteração: 0
Result
0.2718204092236854
Iteração: 1
Result
0.3381131841541069
Iteração: 2
Result
0.34256663952335653
Iteração: 3
Result
0.33794401800042023
Iteração: 4
Result
0.3388475102662005
days_to_recovery: 9
Iteração: 0
Result
0.34643619714439067
Iteração: 1
Result
0.34730739978955183
Iteração: 2
Result
0.3469914779864009
Iteração: 3
Result
0.2842872487866711
Iteração: 4
Result
0.2843140996785702
days_to_recovery: 10
Iteração: 0
Result
0.3591516011937554
Iteração: 1
Result
0.35971400669416886
Iteração: 2
Result
0.36096250823035037
Iteração: 3
Result
0.3614911292276478
Iteração: 4
Result
0.3583820134887846
days_to_recovery: 11
Iteração: 0
Result
0.36877944773616034
Iteração: 1
Result
0.37029776244996054
Iteração: 2
Result
0.3708327289409602
Iteração: 3
Result
0.3688024156188065
Iteração: 4
Result
0.3674644323395223
days_to_recovery: 12
Iteração: 0
Result
0.38464417905553117
Iteração: 1
Result
0.38331384169101257
Iteração: 2
Result
0.382107434277493
Iteração: 3
Re